In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

A user has requested how many seven day prescriptions there are for Aspirin as a proprotion of all prescriptions for Aspirin. The aim is to use as a surrogate for dosette boxes. The Pharmaceutical Journal has a good article here from earlier in the year that questions are we dispensing too many dosette boxes. It relies on small audits in one or two CCGs to extrapolate to the whole country. Dosette boxes are generally not recommended and as all good pharmacy staff will tell you they are a pain in the backside to dispense and assure yourself of the contents.

We will use medicines for long term conditions as a surrogate for indicating dosette boxes as it is once daily so prescriptions with a seven day course length are indicative of weekly scripts which are sometimes used (erroneously) to support dispensing in dosette boxes.

In [3]:
sql='''SELECT
bnf.chemical,
  SUM(CASE
      WHEN quantity_per_item=7 THEN total_quantity
      ELSE 0
  END) AS seven_day_QI,
  SUM(total_quantity) AS all_QI,
  IEEE_DIVIDE(SUM(CASE
      WHEN quantity_per_item=7 THEN total_quantity
      ELSE 0
  END),
      SUM(total_quantity)) AS percent_7_day_QI
FROM
  ebmdatalab.hscic.raw_prescribing_normalised AS presc
INNER JOIN
  hscic.bnf as bnf
ON
  presc.bnf_code=bnf.presentation_code

WHERE
(presentation LIKE '%_Tab%' or presentation LIKE '%_Cap%')
    AND SUBSTR(bnf_code,0,2) IN ('01','02','04','06') ## here we use common BNF chapters. Chapter 3 resp is excluded due to amount of inhalers
GROUP BY
bnf.chemical
ORDER BY
  percent_7_day_QI DESC
    LIMIT 50'''


df_ltc_meds = bq.cached_read(sql, csv_path='aspirin_sevendays.csv')
df_ltc_meds.head(5)

,chemical,seven_day_QI,all_QI,percent_7_day_QI
0,Biperiden Hydrochloride,154,427,0.360656
1,Ambrisentan,119,733,0.162347
2,Donepezil Hydrochloride,15929403,102690416,0.155121
3,Galantamine,1208543,9561130,0.126402
4,Memantine Hydrochloride,6532484,63451228,0.102953
